# Functions

In [1]:
def scraping_data_open_and_games(year, competition, scores=False):
    base_url = "https://c3po.crossfit.com/api/leaderboards/v2/competitions"
    data_list = []
    genders = [1, 2]

    # Iterate over each gender
    for gender in genders:
        # Create the initial API URL to fetch the data for the first page
        url = f"{base_url}/{competition}/{year}/leaderboards?division={gender}&page=1"
        try:
            # Fetch the response for the first page
            response = requests.get(url).json()
            # Extract the total number of pages from the response
            total_pages = response['pagination']['totalPages']
            # Iterate over each page
            for page in range(1, total_pages + 1):
                # Create the API URL for each page
                url = f"{base_url}/{competition}/{year}/leaderboards?division={gender}&page={page}"
                try:
                    # Fetch the response for the current page
                    response = requests.get(url).json()
                    # Extract the data for each row in the leaderboardRows
                    for row in response['leaderboardRows']:
                        # Append entrant data for the first query
                        if not scores:
                            row_data = row['entrant'].copy()
                            row_data['overallRank'] = row['overallRank']
                            row_data['overallScore'] = row['overallScore']
                            # Append the row data to the data_list
                            data_list.append(row_data)
                        # Append score data for the second query
                        else:
                            total_ordinals = pd.DataFrame(row['scores'] for row in response['leaderboardRows']).shape[1]
                            for ordinal in range(0, total_ordinals):
                                score_data = row['scores'][ordinal].copy()
                                score_data['competitorId'] = row['entrant']['competitorId']
                                # Append the score data to the data_list
                                data_list.append(score_data)
                except Exception as e:
                    # Handle any errors that occur during the API request for a specific page
                    print(f"Error occurred while fetching data for gender={gender}, page={page}: {e}")
        except Exception as e:
            # Handle any errors that occur during the API request for fetching total_pages
            print(f"Error occurred while fetching total_pages for gender={gender}: {e}")

    # Create a DataFrame from the collected data_list
    df = pd.DataFrame(data_list)
    
    return df